In [0]:
pip install contractions

Python interpreter will be restarted.
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp38-cp38-linux_x86_64.whl size=98363 sha256=c8d66038334b6d27f71cbaf0918082852539cb5d7226430f9ff58758fe835741
  Stored in directory: /root/.cache/pip/wheels/74/bc/b8/e5f739a84005620cfe66d3fcb8bb182e309d6056bc6700b60e
Successfully built pyahocorasick
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d5108422-b612-4317-8b3e-2872393a2b90/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


### Step 1: Reading the data

In [0]:
reviewsDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/reviews.json"))

reviewsDF.display()

business_id,cool,date,funny,month,stars,text,useful,year
SfkRN960vVQ74mbowSEuXQ,0,2019-01-03T04:12:41.000Z,0,1,5.0,"Affordable prices, fast service, and a friendly dog named Rambo. This place is great.",0,2019-01-01
YZs1gNSh_sN8JmN_nrpxeA,0,2019-01-10T22:14:40.000Z,0,1,5.0,"One of my favorite BBQ places in town. The meats and the sides are both fantastic, it doesn't smell like your table is located inside the smoker, and the noise level is manageable. Highly recommended.",0,2019-01-01
gGvNgShksetPoimyKV8I9Q,0,2019-01-10T01:25:05.000Z,0,1,1.0,I can't rate the food but we walked in were seated. Waited 20 minutes before our waitress approached. Then stated she'd be back to take our drink order. Came back another 10 minutes later and took our drink order. After waiting 20 more minutes and not getting even our drinks we decided to leave. Worst service I've ever had and it wasn't even that busy!,0,2019-01-01
ttZRucfe1fcN_DX2ZGZ1Vg,0,2019-01-14T17:54:56.000Z,0,1,3.0,"This restaurant is conveniently located down the street from the Fox and was an easy spot to grab a quick bite before a show on Sunday. Although the restaurant is Italian it has a fairly limited selection of pasta. The service was inconsistent despite not being extremely crowded. We had to flag down our waitress a few time for refills and bread. Monday - Friday they offer $4 wine! Despite going on a Sunday the bartender was gracious enough to honor this special for me and my friend. I ordered the Gorgonzola salad with chicken which took well over 20 mins to be delivered to the table. The salad was average and a bit overpriced ($20) which may be due to the area. The bruschetta is flavorful and the calamari was well seasoned. All in all the restaurants is a good spot to partake in the $4 wine during the week. The service leaves a little to be desired, however hopefully would be better during the week.",0,2019-01-01
R0IJhEI-zSJpYT1YN_orWQ,0,2019-01-01T00:54:29.000Z,0,1,5.0,"This nail salon is amazing. Nikki, Thao, Kathy and Anna are all very talented and really listen to the customers. This salon has an amazing amount of colors- the most I've seen in the area. They also stay up with all the trends out there so they have fun designs! Love this place!",0,2019-01-01
qDyZQ6mT1JajQ7_Yj7PeLw,1,2019-01-03T21:14:24.000Z,0,1,4.0,"""Food - 4.5 stars Service - 3.5 stars I'm going to average it out at 4 stars. It really is a great spot, especially if you're shopping at the domain and want to drop in at non-peak times, since it will get pretty busy! I love the option to sit at the bar if you've got a small party (1-2) where it's first come first serve. It's our go-to spot when we're craving Japanese beer, which they have on tap! My favorite is the spicy chicken ramen, which I get at the spiciest level - good heat, nice veggies, some umami flavor, thin noodles. We've had a number of the appetizers here as well, standouts being the brussel sprouts and crispy chicken. They've also got great combos, where you can add on things like crispy chicken + salad to a ramen order, which my fiance and I will often split as some food to go with our beers. My only real complaint is the service, which leaves something to be desired. We'll often have to ask for things twice, or go hunt down waiters to ask for more water, napkins, etc. Once, we ordered drinks, and only got it after we waited 20 minutes for our food. They also haven't been the friendliest staff we've experienced, though I get the feeling that a lot of times they're just feeling slammed/overwhelmed by the high volume of customers. Still, we've had very pleasant experiences at other crowded restaurants, so I don't think it's unreasonable to want a friendlier/more helpful staff here. Overall, the food and the beer/hot sake on tap makes this worthwhile to go to. Try to go during happy hour/off times if you can!""",1,2019-01-01
X7ZPCV8jyS5rw_bI0aWNNA,0,2019-01-05T03:44:43.000Z,0,1,5.0,"Rented a local spot for a large family reunion. Convenientl

In [0]:
reviewsDF.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- month: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- year: string (nullable = true)



### Step 2: Data Preparation

### Step 2.1: Filter out non-f&b reviews

In [0]:
bizDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/dataset_business_cleaned.json"))

biz_pdf=bizDF.toPandas()
biz_id_list=biz_pdf['business_id'].unique()
biz_id_list

Out[3]: array(['6iYb2HFDywm3zjuRg0shjw', 'tCbdrRPZA0oiIYSmHG3J0w',
       'D4JtQNTI4X3KcbzacDJsMw', ..., 'bQX-kwVTyZgcdZGEPzce6Q',
       'GB75wPibj3IjNauaoCxyGA', 'ngmLL5Y5OT-bYHKU0kKrYA'], dtype=object)

In [0]:
reviews_pdf=reviewsDF.select("*").toPandas()
reviews_pdf

,business_id,cool,date,funny,month,stars,text,useful,year
0,SfkRN960vVQ74mbowSEuXQ,0,2019-01-03T04:12:41.000Z,0,1,5.0,"Affordable prices, fast service, and a friendl...",0,2019-01-01
1,YZs1gNSh_sN8JmN_nrpxeA,0,2019-01-10T22:14:40.000Z,0,1,5.0,One of my favorite BBQ places in town. The mea...,0,2019-01-01
2,gGvNgShksetPoimyKV8I9Q,0,2019-01-10T01:25:05.000Z,0,1,1.0,I can't rate the food but we walked in were se...,0,2019-01-01
3,ttZRucfe1fcN_DX2ZGZ1Vg,0,2019-01-14T17:54:56.000Z,0,1,3.0,This restaurant is conveniently located down t...,0,2019-01-01
4,R0IJhEI-zSJpYT1YN_orWQ,0,2019-01-01T00:54:29.000Z,0,1,5.0,"This nail salon is amazing. Nikki, Thao, Kathy...",0,2019-01-01
...,...,...,...,...,...,...,...,...,...
1639455,O1pWzMvVmBomQUaZ3aUlLw,1,2020-12-30T18:38:38.000Z,0,12,5.0,This place is legit! I had the Funky Monkey Fr...,1,2020-01-01
1639456,HpiE8X3s8xRRRihjp7_IMA,0,2020-11-24T01:06:39.000Z,0,11,5.0,this place is amazing! we ate tiramisu and str...,0,2020-01-01
1639457,kwHcdJttuelU1TTZVDovbw,0,2020-12-25T09:41:16.000Z,0,12,5.0,"""My family and I are longtime patrons of TOC, ...",0,2020-01-01
1639458,gEQxTJDoJYaW0l_6FYtf8g,0,2020-12-05T21:32:45.000Z,0,12,5.0,Best pizza in the neighborhood!!! Love the thi...,1,2020-01-01


In [0]:
reviews_pdf.shape

Out[5]: (1639460, 9)

In [0]:
reviews_pdf=reviews_pdf[reviews_pdf['business_id'].isin(biz_id_list)]

In [0]:
reviews_pdf.shape

Out[7]: (1162914, 9)

### Step 2.1: Removing missing values

In [0]:
reviews_pdf.isnull().sum()

Out[8]: business_id      0
cool             0
date           258
funny            0
month            0
stars            0
text             0
useful           0
year             0
dtype: int64

In [0]:
reviews_pdf=reviews_pdf.dropna()
reviews_pdf.head()

,business_id,cool,date,funny,month,stars,text,useful,year
1,YZs1gNSh_sN8JmN_nrpxeA,0,2019-01-10T22:14:40.000Z,0,1,5.0,One of my favorite BBQ places in town. The mea...,0,2019-01-01
2,gGvNgShksetPoimyKV8I9Q,0,2019-01-10T01:25:05.000Z,0,1,1.0,I can't rate the food but we walked in were se...,0,2019-01-01
3,ttZRucfe1fcN_DX2ZGZ1Vg,0,2019-01-14T17:54:56.000Z,0,1,3.0,This restaurant is conveniently located down t...,0,2019-01-01
5,qDyZQ6mT1JajQ7_Yj7PeLw,1,2019-01-03T21:14:24.000Z,0,1,4.0,"""Food - 4.5 stars\nService - 3.5 stars\n\nI'm ...",1,2019-01-01
8,Y2cj0TQvfSdRU3H55CkEgA,0,2019-01-02T19:25:48.000Z,0,1,1.0,service was awful. charged for wrong size and ...,0,2019-01-01


In [0]:
reviews_pdf.isnull().sum()

Out[10]: business_id    0
cool           0
date           0
funny          0
month          0
stars          0
text           0
useful         0
year           0
dtype: int64

In [0]:
reviews_pdf.reset_index(inplace=True)

In [0]:
reviews_pdf=reviews_pdf.set_index("index")

### Step 2.2: Text Cleaning

In [0]:
#remove url
reviews_pdf['text_basic_cleaned'] = reviews_pdf['text'].str.replace('http\S+|www.\S+|https\S+', '', case=False)

<command-1095695663389510>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews_pdf['text_basic_cleaned'] = reviews_pdf['text'].str.replace('http\S+|www.\S+|https\S+', '', case=False)


In [0]:
#remove punctutation
import re

reviews_pdf['text_basic_cleaned'] = reviews_pdf['text_basic_cleaned'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [0]:
#remove special characters
import string
def text_cleaner(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text)
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[0-9]*", "", text)
    text = re.sub(r"(”|“|-|\+|`|#|,|;|\|)*", "", text)
    text = re.sub(r"&amp", "", text)
    text = text.lower()
        
    return text
reviews_pdf['text_basic_cleaned']=reviews_pdf['text_basic_cleaned'].apply(text_cleaner)
reviews_pdf['text_basic_cleaned']=reviews_pdf['text_basic_cleaned'].str.replace('[{}]'.format(string.punctuation), '')
reviews_pdf

<command-1095695663389512>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews_pdf['text_basic_cleaned']=reviews_pdf['text_basic_cleaned'].str.replace('[{}]'.format(string.punctuation), '')


,business_id,cool,date,funny,month,stars,text,useful,year,text_basic_cleaned
index,,,,,,,,,,
1,YZs1gNSh_sN8JmN_nrpxeA,0,2019-01-10T22:14:40.000Z,0,1,5.0,One of my favorite BBQ places in town. The mea...,0,2019-01-01,one of my favorite bbq places in town the meat...
2,gGvNgShksetPoimyKV8I9Q,0,2019-01-10T01:25:05.000Z,0,1,1.0,I can't rate the food but we walked in were se...,0,2019-01-01,i cant rate the food but we walked in were sea...
3,ttZRucfe1fcN_DX2ZGZ1Vg,0,2019-01-14T17:54:56.000Z,0,1,3.0,This restaurant is conveniently located down t...,0,2019-01-01,this restaurant is conveniently located down t...
5,qDyZQ6mT1JajQ7_Yj7PeLw,1,2019-01-03T21:14:24.000Z,0,1,4.0,"""Food - 4.5 stars\nService - 3.5 stars\n\nI'm ...",1,2019-01-01,food stars\nservice stars\n\nim going to a...
8,Y2cj0TQvfSdRU3H55CkEgA,0,2019-01-02T19:25:48.000Z,0,1,1.0,service was awful. charged for wrong size and ...,0,2019-01-01,service was awful charged for wrong size and w...
...,...,...,...,...,...,...,...,...,...,...
1639455,O1pWzMvVmBomQUaZ3aUlLw,1,2020-12-30T18:38:38.000Z,0,12,5.0,This place is legit! I had the Funky Monkey Fr...,1,2020-01-01,this place is legit i had the funky monkey fre...
1639456,HpiE8X3s8xRRRihjp7_IMA,0,2020-11-24T01:06:39.000Z,0,11,5.0,this place is amazing! we ate tiramisu and str...,0,2020-01-01,this place is amazing we ate tiramisu and stra...
1639457,kwHcdJttuelU1TTZVDovbw,0,2020-12-25T09:41:16.000Z,0,12,5.0,"""My family and I are longtime patrons of TOC, ...",0,2020-01-01,my family and i are longtime patrons of toc an...


In [0]:
#expand contractions
import contractions

def expand(text):
    # creating an empty list
    expanded_words = []    
    for word in text.split():
        # using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    return expanded_text

reviews_pdf['text_basic_cleaned']=reviews_pdf['text_basic_cleaned'].apply(expand)

### Step 2.3: Splitting Timestamp column

In [0]:
reviews_pdf.columns

Out[17]: Index(['business_id', 'cool', 'date', 'funny', 'month', 'stars', 'text',
       'useful', 'year', 'text_basic_cleaned'],
      dtype='object')

In [0]:
import pandas as pd
reviews_pdf["date"]=pd.to_datetime(reviews_pdf["date"])
reviews_pdf["date"]=reviews_pdf['date'].dt.date

In [0]:
reviews_pdf.drop(['year'],axis=1,inplace=True)

In [0]:
'''
import pandas as pd
import datetime as dt
reviews_pdf['Year'] = reviews_pdf['date'].dt.year 
reviews_pdf['Month'] = reviews_pdf['date'].dt.month 
reviews_pdf['Day'] = reviews_pdf['date'].dt.day 
reviews_pdf['DayOfWeek'] = reviews_pdf['date'].dt.dayofweek

'''


Out[19]: "\nimport pandas as pd\nimport datetime as dt\nreviews_pdf['Year'] = reviews_pdf['date'].dt.year \nreviews_pdf['Month'] = reviews_pdf['date'].dt.month \nreviews_pdf['Day'] = reviews_pdf['date'].dt.day \nreviews_pdf['DayOfWeek'] = reviews_pdf['date'].dt.dayofweek\n\n"

In [0]:
reviews_pdf

,business_id,cool,date,funny,month,stars,text,useful,text_basic_cleaned
index,,,,,,,,,
1,YZs1gNSh_sN8JmN_nrpxeA,0,2019-01-10,0,1,5.0,One of my favorite BBQ places in town. The mea...,0,one of my favorite bbq places in town the meat...
2,gGvNgShksetPoimyKV8I9Q,0,2019-01-10,0,1,1.0,I can't rate the food but we walked in were se...,0,i cannot rate the food but we walked in were s...
3,ttZRucfe1fcN_DX2ZGZ1Vg,0,2019-01-14,0,1,3.0,This restaurant is conveniently located down t...,0,this restaurant is conveniently located down t...
5,qDyZQ6mT1JajQ7_Yj7PeLw,1,2019-01-03,0,1,4.0,"""Food - 4.5 stars\nService - 3.5 stars\n\nI'm ...",1,food stars service stars I am going to average...
8,Y2cj0TQvfSdRU3H55CkEgA,0,2019-01-02,0,1,1.0,service was awful. charged for wrong size and ...,0,service was awful charged for wrong size and w...
...,...,...,...,...,...,...,...,...,...
1639455,O1pWzMvVmBomQUaZ3aUlLw,1,2020-12-30,0,12,5.0,This place is legit! I had the Funky Monkey Fr...,1,this place is legit i had the funky monkey fre...
1639456,HpiE8X3s8xRRRihjp7_IMA,0,2020-11-24,0,11,5.0,this place is amazing! we ate tiramisu and str...,0,this place is amazing we ate tiramisu and stra...
1639457,kwHcdJttuelU1TTZVDovbw,0,2020-12-25,0,12,5.0,"""My family and I are longtime patrons of TOC, ...",0,my family and i are longtime patrons of toc an...


### Step 3: Convert back to spark and save

In [0]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
reviewsDF=spark.createDataFrame(reviews_pdf) 





In [0]:
reviewsDF.coalesce(1).write.mode("overwrite").format('json').save("/FileStore/tables/Project_Data_Extract/dataset_reviews_cleaned.json")

In [0]:
# accessing cleaned file
reviewsDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/dataset_reviews_cleaned.json"))

reviewsDF.display()

business_id,cool,date,funny,month,stars,text,text_basic_cleaned,useful
YZs1gNSh_sN8JmN_nrpxeA,0,2019-01-10,0,1,5.0,"One of my favorite BBQ places in town. The meats and the sides are both fantastic, it doesn't smell like your table is located inside the smoker, and the noise level is manageable. Highly recommended.",one of my favorite bbq places in town the meats and the sides are both fantastic it does not smell like your table is located inside the smoker and the noise level is manageable highly recommended,0
gGvNgShksetPoimyKV8I9Q,0,2019-01-10,0,1,1.0,I can't rate the food but we walked in were seated. Waited 20 minutes before our waitress approached. Then stated she'd be back to take our drink order. Came back another 10 minutes later and took our drink order. After waiting 20 more minutes and not getting even our drinks we decided to leave. Worst service I've ever had and it wasn't even that busy!,i cannot rate the food but we walked in were seated waited minutes before our waitress approached then stated she would be back to take our drink order came back another minutes later and took our drink order after waiting more minutes and not getting even our drinks we decided to leave worst service I have ever had and it was not even that busy,0
ttZRucfe1fcN_DX2ZGZ1Vg,0,2019-01-14,0,1,3.0,"This restaurant is conveniently located down the street from the Fox and was an easy spot to grab a quick bite before a show on Sunday. Although the restaurant is Italian it has a fairly limited selection of pasta. The service was inconsistent despite not being extremely crowded. We had to flag down our waitress a few time for refills and bread. Monday - Friday they offer $4 wine! Despite going on a Sunday the bartender was gracious enough to honor this special for me and my friend. I ordered the Gorgonzola salad with chicken which took well over 20 mins to be delivered to the table. The salad was average and a bit overpriced ($20) which may be due to the area. The bruschetta is flavorful and the calamari was well seasoned. All in all the restaurants is a good spot to partake in the $4 wine during the week. The service leaves a little to be desired, however hopefully would be better during the week.",this restaurant is conveniently located down the street from the fox and was an easy spot to grab a quick bite before a show on sunday although the restaurant is italian it has a fairly limited selection of pasta the service was inconsistent despite not being extremely crowded we had to flag down our waitress a few time for refills and bread monday friday they offer wine despite going on a sunday the bartender was gracious enough to honor this special for me and my friend i ordered the gorgonzola salad with chicken which took well over mins to be delivered to the table the salad was average and a bit overpriced which may be due to the area the bruschetta is flavorful and the calamari was well seasoned all in all the restaurants is a good spot to partake in the wine during the week the service leaves a little to be desired however hopefully would be better during the week,0
qDyZQ6mT1JajQ7_Yj7PeLw,1,2019-01-03,0,1,4.0,"""Food - 4.5 stars Service - 3.5 stars I'm going to average it out at 4 stars. It really is a great spot, especially if you're shopping at the domain and want to drop in at non-peak times, since it will get pretty busy! I love the option to sit at the bar if you've got a small party (1-2) where it's first come first serve. It's our go-to spot when we're craving Japanese beer, which they have on tap! My favorite is the spicy chicken ramen, which I get at the spiciest level - good heat, nice veggies, some umami flavor, thin noodles. We've had a number of the appetizers here as well, standouts being the brussel sprouts and crispy chicken. They've also got great combos, where you can add on things like crispy chicken + salad to a ramen order, which my fiance and I will often split as some food to go with our beers. My only real complain

In [0]:
dbutils.fs.rm("/FileStore/tables/Project_Data_Extract/reviews.json",True)

Out[32]: True